<a href="https://colab.research.google.com/github/Suryanshi2003/Data_Science_projects/blob/main/SkinCancerDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm



In [ ]:
import os

os.environ["KAGGLE_USERNAME"]="suryanshi15"
os.environ["KAGGLE_KEY"]="19fc2570419575e80c41e6922242ded5"
!kaggle datasets download nodoubttome/skin-cancer9-classesisic

In [ ]:
!unzip skin-cancer9-classesisic.zip

Archive:  skin-cancer9-classesisic.zip
replace Skin cancer ISIC The International Skin Imaging Collaboration/Test/actinic keratosis/ISIC_0010512.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_path="dataset/Train"
test_path="dataset/Test"

In [ ]:
class_names=os.listdir(train_path)
class_names

In [ ]:
for name in class_names:
  path=train_path+'/'+ name
  n_images=len(os.listdir(path))
  print(name,' : ', n_images)

In [ ]:
for name in class_names:
  path=test_path+'/'+ name
  n_images=len(os.listdir(path))
  print(name,' : ', n_images)

In [ ]:
random_img="dataset/Train/dermatofibroma/ISIC_0025314.jpg"

In [ ]:
img=cv2.imread(random_img)


In [ ]:
img.shape

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
def load_data(path):
  images_list=[]
  labels_list=[]
  for i in range(len(class_names)):
    dir_path=path+ "/" +class_names[i]
    images= os.listdir(dir_path)
    for j in tqdm(range(len(images))):
      img_path=dir_path + "/"+ images[j]
      img=cv2.imread(img_path)
      img=cv2.resize(img,(224,224))
      images_list.append(img/255.0)
      labels_list.append(i)
  images_list=np.asarray(images_list)
  labels_list=np.asarray(labels_list)
  return images_list,labels_list

In [ ]:
train_images,train_labels= load_data(train_path)

In [ ]:
train_images.shape

In [ ]:
test_images,test_labels = load_data(test_path)

In [ ]:

plt.imshow(train_images[564])
plt.show()

In [ ]:
##Data preprocessing
train_images[0][0][0]

In [ ]:
train_images[0][0][0]

In [ ]:
#to train data from tensorflow ,first convert to tensor object not array
BUFFER_SIZE=500
BATCH_SIZE=32

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten
from tensorflow.keras.layers import LeakyReLU,Dropout,BatchNormalization
from tensorflow.keras.models import Model

In [ ]:

train_df=tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_df=tf.data.Dataset.from_tensor_slices((test_images,test_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train_df

#This is second type of model building in tensorflow

In [ ]:

#first we inherit model class
class MyModel(Model):
  #_init_-constructor
  def __init__(self):
    #calling parent class constructor
    super().__init__()
    #first convolutional layer
    #32 is no.of filters
    #3 is kernel size
    self.conv_1_1=Conv2D(32,3,activation='relu',input_shape=(299,299,3))
    self.conv_1_2=Conv2D(32,3,activation='relu',input_shape=(299,299,3))
    #maxpool layer
    self.pool_1= MaxPool2D(pool_size=(2,2))
    self.conv_2_1=Conv2D(32,3,activation='relu')
    self.conv_2_2=Conv2D(32,3,activation='relu')
    #maxpool layer
    self.pool_2= MaxPool2D(pool_size=(2,2))
    self.conv_3_1=Conv2D(32,3,activation='relu')
    self.conv_3_2=Conv2D(32,3,activation='relu')
    #maxpool layer
    self.pool_3= MaxPool2D(pool_size=(2,2))
    #Apply flatten layer
    self.flatten=Flatten()
    #hidden layer
    self.hidden_1=Dense(512)
    self.activation=LeakyRelu()
    self.dropout_1=Dropout(0.3)
    self.batch_1=BatchNormalization()
    self.hidden_2=Dense(512)
    self.dropout_2=Dropout(0.3)
    self.batch_2=BatchNormalization()
    self.hidden_3=Dense(128)
    self.dropout_3=Dropout(0.3)
    self.batch_3=BatchNormalization()
    self.hidden_4=Dense(64)
    self.dropout_4=Dropout(0.3)
    self.batch_4=BatchNormalization()
    #Output layer
    self.outputlayer=Dense(9,activation='softmax')

  def call(self,data):
    x=self.conv_1_1(data)
    x=self.conv_1_2(data)
    x=self.pool_1(x)
    x=self.conv_2_1(x)
    x=self.conv_2_2(x)
    x=self.pool_2(x)
    x=self.conv_3_1(x)
    x=self.conv_3_2(x)
    x=self.pool_3(x)
    x=self.flatten(x)
    x=self.hidden_1(x)
    x=self.activation(x)
    x=self.dropout_1(x)
    x=self.batch_1(x)
    x=self.hidden_2(x)
    x=self.activation(x)
    x=self.dropout_2(x)
    x=self.batch_2(x)
    x=self.hidden_3(x)
    x=self.activation(x)
    x=self.dropout_3(x)
    x=self.batch_3(x)
    x=self.hidden_4 (x)
    x=self.activation(x)
    x=self.dropout_4(x)
    x=self.batch_4(x)

    return self.outputlayer(x)

In [ ]:
model=MyModel()

In [ ]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.RMSprop()
train_acc=tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
test_acc=tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

train_loss=tf.keras.metrics.Mean(name='train_loss')
test_loss=tf.keras.metrics.Mean(name='test_loss')

In [ ]:
#Decorators- we pass function as an arguement in another function
#here @tf.function is a decorator
#Now it means that there is a  function in tensorflow whose name is function
#now when model starts to train then train_step function will be passed as an arguement inside decorator
@tf.function
def train_step(train_images,train_labels):
  #now once we start tensorflow graph it will initiate process of gradient descent
  with tf.GradientTape() as tape:
    #it will execute call that we define in our class
    predictions=model(train_images)
    #now we will calculate loss on our training data
    loss=loss_object(train_labels,predictions)

    #optimization
    #it will keep track of gradients for your computers
    grad=tape.gradient(loss,model.trainable_variables)
    #apply optimization- it will update weights and bias
    optimizer.apply_gradients(zip(grad,model.trainable_variables))

    train_loss(loss)
    train_acc(train_labels,predictions)

In [ ]:
@tf.function
def test_step(test_images,test_labels):
  predictions=model(test_images)
  loss=loss_object(test_labels,predictions)
  test_loss(loss)
  test_acc(test_labels,predictions)


In [ ]:
epochs=50
for epoch in range(epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()
  for images,labels in train_df:
    train_step(images,labels)
  for images,labels in test_df:
    test_step(images,labels)

  print("Epoch : {} | Train Loss : {:.3f} | Train Acc : {:.3f} | Test Loss : {} | Test Acc {}".format(
                                                                                              epochs,train_loss.result(),
                                                                                              train_acc.result(),
                                                                                              test_loss.result(),
                                                                                              test_acc.result()))